# Training with PyTorch
This notebook shows how to train a PyTorch model using images from `vl-datasets`. 

<!--<badge>--><a href="https://colab.research.google.com/github/visual-layer/vl-datasets/blob/main/notebooks/train-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->
<!--<badge>--><a href="https://kaggle.com/kernels/welcome?src=https://github.com/visual-layer/vl-datasets/blob/main/notebooks/train-pytorch.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/></a><!--</badge>-->

## Installation & Setting Up

In [ ]:
!pip install vl-datasets -Uqq

In [1]:
import vl_datasets
vl_datasets.__version__

'0.0.5'

## Download food-101 Dataset

The easiest way to get the clean version of food-101 dataset is to use `vl_dataset` package.

In [2]:
from vl_datasets import CleanFood101

train_dataset = CleanFood101('./', split='train')
valid_dataset = CleanFood101('./', split='test')

Excluded 407 from the train set
Excluded 91 from the test set


Upon completion, you'd have with a `food-101` folder in your local directory.

```
food-101/
├── images
│   ├── apple_pie
│   │   ├── 0001.jpg
│   │   └── 0002.jpg
│   ├── baby_back_ribs
│   ├── baklava
│   ├── ...
│   ├── ...
│   ├── ...
│   └── waffles
├── meta
|   ├── classes.txt
│   ├── labels.txt
│   ├── test.json
│   ├── test.txt
│   ├── train.json
│   └── train.txt
├── license_agreement.txt
└── README.txt
```


If you'd like to download the food-101 manually, you run the following codes to download and extract the folder into your local directory.

```shell
wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
tar -xf food-101.tar.gz
```

Next, load the directory into a `CleanFood101` class.

```python
train_dataset = CleanFood101('./', download=False, split='train', exclude_csv='food-101.csv.csv')
valid_dataset = CleanFood101('./', download=False, split='test', exclude_csv='food-101.csv.csv')
```

Download the .csv file for food-101 dataset [here](https://drive.google.com/uc?export=download&id=1ZG5GvU342l4YmSeYo6v6LeKbMM5fwjjw).

View the first five problematic images that are listed in the `.csv` file.

In [4]:
print(train_dataset.excluded_files[:5])

['churros/1440917.jpg', 'churros/1944265.jpg', 'churros/3867252.jpg', 'churros/644700.jpg', 'hot_and_sour_soup/1674538.jpg']


## Import PyTorch and Torchvision

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

We can also view the exclude files with:

In [6]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True)

Adjust the `batch_size` to a value that fits your hardware.

## Define the model architecture
Let's construct a basic convolutional model, `Resnet18` from `Torchvision`.

In [7]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

## Define the loss function and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model
Now, let's write a simple training loop to train the model for 5 epochs on a GPU or CPU.

In [9]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")


Using device: cuda
Epoch 1 - Loss: 2.39756304611594
Epoch 2 - Loss: 1.802627974445537
Epoch 3 - Loss: 1.6139094174918482
Epoch 4 - Loss: 1.4799583055205265
Epoch 5 - Loss: 1.3918423923395449


## Evaluate the model
Finally we evaluate the model on the validation set and prints it's accuracy.

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")


Accuracy: 72.91227791247665
